# **(Cybersecurity Threat Detection ETL)**

## Objectives

Load the raw Kaggle intrusion detection CSV(s)
-  Load the raw data from DataSet>Raw folder
- Clean data: Looks for missing values, duplicates, and inconsistent data types
- Standardize column names (lowercase, underscores instead of spaces/special characters)
- Convert categorical variables to appropriate types (e.g., one-hot encoding or label encoding)
- Remove duplicates and irrelevant columns (e.g., IDs, timestamps if not needed)
- Handle outliers (e.g., capping, removal, or transformation)
- Normalize/scale numerical features if necessary (e.g., Min-Max scaling, Standardization)
- Impute missing values (numeric=median, categorical=mode) to produce a BI-ready table
- Export tidy CSVs to DataSet>Processed folder

## Inputs

* Raw Data: DataSet>Raw>Test_data.csv

## Outputs

* Processed Data: DataSet>Processed>Processed_Test_data.csv

## Additional Comments





---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [3]:
import os
current_dir = os.getcwd()
current_dir

'c:\\Users\\Nine\\OneDrive\\Documents\\VS Code Projects\\CyberSec-Threat-Detection\\jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [4]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [5]:
current_dir = os.getcwd()
current_dir

'c:\\Users\\Nine\\OneDrive\\Documents\\VS Code Projects\\CyberSec-Threat-Detection'

# Section 1

Import libraries

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

---

# Section 2

Load the raw data and start ETL process.

In [7]:
# Load the raw dataset
df = pd.read_csv('DataSet/Raw/Test_data.csv')

# Display basic information about the dataset
print(f"Dataset shape: {df.shape}")
print(f"Number of rows: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")
print("\nFirst few rows:")
df.head()

Dataset shape: (22544, 41)
Number of rows: 22544
Number of columns: 41

First few rows:


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,tcp,private,REJ,0,0,0,0,0,0,...,255,10,0.04,0.06,0.00,0.00,0.0,0.0,1.00,1.00
1,0,tcp,private,REJ,0,0,0,0,0,0,...,255,1,0.00,0.06,0.00,0.00,0.0,0.0,1.00,1.00
2,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,134,86,0.61,0.04,0.61,0.02,0.0,0.0,0.00,0.00
3,0,icmp,eco_i,SF,20,0,0,0,0,0,...,3,57,1.00,0.00,1.00,0.28,0.0,0.0,0.00,0.00
4,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,29,86,0.31,0.17,0.03,0.02,0.0,0.0,0.83,0.71


---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* In cases where you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  # create your folder here
  # os.makedirs(name='')
except Exception as e:
  print(e)
